In [2]:
import sys, os
import copy
import re

import pickle as pkl
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon

# Load Data

In [ ]:

MIN_NUM_READS = 100

XCRISP_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/X-CRISP/XCRISP_mse_0.05__{}.pkl"
XCRISP_KLD_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/X-CRISP/XCRISP_kld_0.01__{}.pkl"
LINDEL_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/Lindel/Lindel_{}.pkl"
FORECasT_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/FORECasT/predictions_{}x_{}.pkl"
INDELPHI_PREDICTIONS_F = "/Users/colm/repos/output/local/model_predictions/inDelphi/{}_predictions.pkl"
# TEST_FILES = ["test", "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1", "052218-U2OS-+-LibA-postCas9-rep1_transfertest", "0226-PRLmESC-Lib1-Cas9_transfertest", "TREX_A_test", "HAP1_test"]
TEST_FILES = ["test", "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1"]
BASELINE_TEST_FILES = ["test", "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1"]
data = {}

for i, t in enumerate(TEST_FILES):
    data[t] = {}
    data[t]["X-CRISP-MSE"] = pkl.load(open(XCRISP_PREDICTIONS_F.format(t), 'rb'))
    data[t]["X-CRISP-KLD"] = pkl.load(open(XCRISP_KLD_PREDICTIONS_F.format(t), 'rb'))
    data[t]["inDelphi"] = pkl.load(open(INDELPHI_PREDICTIONS_F.format(t), 'rb'))
    data[t]["Lindel"] = pkl.load(open(LINDEL_PREDICTIONS_F.format(t), 'rb'))
    data[t]["FORECasT"] = pkl.load(open(FORECasT_PREDICTIONS_F.format(MIN_NUM_READS, t), 'rb'))

    # need to remove psuedocounts that were added to profiles in preparation for training
    for target_site in data[t]["FORECasT"].keys():
        data[t]["FORECasT"][target_site]["actual"] = np.array(data[t]["FORECasT"][target_site]["actual"]) - 0.5

models = list(data[t].keys())
print(models, "loaded")

['X-CRISP-MSE', 'X-CRISP-KLD', 'inDelphi', 'Lindel', 'FORECasT'] loaded


In [5]:
t = "052218-U2OS-+-LibA-postCas9-rep1_transfertest"
method = "X-CRISP-MSE"
target_site = "0_0_0_0_AAATATCTTTAACCTAAAAC"
print(t, method, target_site)

052218-U2OS-+-LibA-postCas9-rep1_transfertest X-CRISP-MSE 0_0_0_0_AAATATCTTTAACCTAAAAC


In [6]:
for t in TEST_FILES:
    for m in models:
        target_site = list(data[t][m].keys())[0]
        for k in data[t][m][target_site].keys():
            print(t, m, target_site, len(data[t][m][target_site][k]))

test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-MSE Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test X-CRISP-KLD Oligo_43170 434
test inDelphi Oligo_43170 107
test inDelphi Oligo_43170 107
test inDelphi Oligo_43170 107
test inDelphi Oligo_43170 107
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test Lindel Oligo_43170 557
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
test FORECasT Oligo_43170 398
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 0_0_0_0_CTTTCACTTTATAGATTTAT 393
0105-mESC-Lib1-Cas9-Tol2-BioRep2-tec

### Reformat files

In [7]:
for t in TEST_FILES:
    for e in data[t]:
        print(t, e, len(data[t][e].keys()))

# collect targets common to all experiments

from functools import reduce
common_oligos = {}


for t in TEST_FILES:
    ts = []
    ts.append(list(data[t]["Lindel"].keys()))
    ts.append(list(data[t]["inDelphi"].keys()))
    ts.append(list(data[t]["FORECasT"].keys()))
    common_oligos[t] = reduce(np.intersect1d, ts)
    print(f"There are {len(common_oligos[t])} oligos common to {t}")

# reformat FORECasT indels
def FC_indel_to_our(fc):
    parts = fc.split("_")
    t = parts[0][0] # type I or D 
    l = int(parts[0][1:]) # size
    p = int(parts[1].split("R")[1]) # start
    if t == "D":
        return "{}+{}".format(p-l, l)
    else:
        # return "I{}".format(l)
        return fc

def inDelphi_to_our(ind):
    if ind[-1] in "ACGT":
        return "1+" + ind[-1]
    if ind.isnumeric():
        return "DL" + ind 
    return ind

for t in TEST_FILES:
    for o in common_oligos[t]:
        if "FORECasT" in data[t]:
            data[t]["FORECasT"][o]["indels"] = [FC_indel_to_our(i) for i in data[t]["FORECasT"][o]["indels"]]
        if "inDelphi" in data[t]:
            data[t]["inDelphi"][o]["indels"] = [inDelphi_to_our(i) for i in data[t]["inDelphi"][o]["indels"]]

# np.seterr(all='raise')
file_mapping = {
    "test": "FORECasT WT",
    "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1": "inDelphi WT",
    "WT": "LUMC WT",
    "0226-PRLmESC-Lib1-Cas9_transfertest": "inDelphi NHEJ-deficient",
    "052218-U2OS-+-LibA-postCas9-rep1_transfertest": "inDelphi USO2 WT",
    "HAP1_test": "FORECasT HAP1",
    "TREX_A_test": "FORECasT TREX",
    "2A_TREX_A_test": "2A_FORECasT TREX",
}

test X-CRISP-MSE 3963
test X-CRISP-KLD 3963
test inDelphi 3739
test Lindel 3963
test FORECasT 3939
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-MSE 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 X-CRISP-KLD 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 inDelphi 1957
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 Lindel 1961
0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1 FORECasT 1961
There are 3738 oligos common to test
There are 1957 oligos common to 0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1


### Generate ins mappings to FORECasT output 

In [8]:
import re
sys.path.append("../modelling")
from src.data.data_loader import get_details_from_fasta

# test over a common set of indels per target site
def generate_1_and_2_bp_insertions():
    nucs = ["A", "C", "G", "T"]
    onebps = []
    twobps = []
    for n1 in nucs:
        onebps.append("1+" + n1)
        for n2 in nucs:
            twobps.append("2+" + n1 + n2)
    return onebps + twobps

common_insertions = generate_1_and_2_bp_insertions()

fasta_files = ["../../src/data/FORECasT/test.fasta", "../../src/data/inDelphi/LibA.fasta"]
guides = {}

for ff in fasta_files:
    guides.update(get_details_from_fasta(ff))

ins_mapping = {}
t = "test"
for t in TEST_FILES:
    ins_mapping[t] = {}
    for o in common_oligos[t]:
        g = guides[o]
        cutsite = g["PAM Index"] - 3
        ins_mapping[t][o] = {}
        FORECasT_insertions = [i for i in data[t]["FORECasT"][o]["indels"] if "I" in i]
        FORECasT_rep_insertions = [i for i in FORECasT_insertions if "C" in i]
        FORECasT_norep_insertions = [i for i in FORECasT_insertions if "C" not in i]
        if len(FORECasT_norep_insertions) not in [1, 2]: print("wtf")
        for i in FORECasT_rep_insertions:
            _, I, _, L, C, R = re.split("I|_|L|C|R", i)
            rep_nuc = g["TargetSequence"][cutsite + int(R) -int(I):cutsite + int(R)]
            ins_mapping[t][o][i] = "{}+{}".format(int(I), rep_nuc)
        for i in FORECasT_norep_insertions:
            if i[1] == "1":
                ins_mapping[t][o][i] = list(np.setdiff1d([c for c in common_insertions if "1" in c], list(ins_mapping[t][o].values()))) 
            if i[1] == "2":
                ins_mapping[t][o][i] = list(np.setdiff1d([c for c in common_insertions if "2" in c], [c for c in list(ins_mapping[t][o].values()) if "2" in c])) 
print("mapped output to FORECasT")
print(ins_mapping[t][o])

mapped output to FORECasT
{'I1_L-1C1R1': '1+A', 'I1_L-2C1R0': '1+T', 'I2_L-1C1R1': '2+TA', 'I2_L-1C2R2': '2+AG', 'I2_L-2C1R0': '2+CT', 'I2_L-2C2R1': '2+TA', 'I2_L-3C2R0': '2+CT', 'I1_L-1R0': ['1+C', '1+G'], 'I2_L-1R0': ['2+AA', '2+AC', '2+AT', '2+CA', '2+CC', '2+CG', '2+GA', '2+GC', '2+GG', '2+GT', '2+TC', '2+TG', '2+TT']}


In [9]:
rev_ins_mapping = {}
for t in ins_mapping:
    rev_ins_mapping[t] = {}
    for o in ins_mapping[t]:
        rev_ins_mapping[t][o] = {}
        for i in ins_mapping[t][o]:
            a = ins_mapping[t][o][i]
            if isinstance(a, list):
                for a2 in a:
                    rev_ins_mapping[t][o][a2] = i
            else:
               rev_ins_mapping[t][o][a] = i
print("Reversed mapping")

rev_ins_mapping[t][o]

Reversed mapping


{'1+A': 'I1_L-1C1R1',
 '1+T': 'I1_L-2C1R0',
 '2+TA': 'I2_L-2C2R1',
 '2+AG': 'I2_L-1C2R2',
 '2+CT': 'I2_L-3C2R0',
 '1+C': 'I1_L-1R0',
 '1+G': 'I1_L-1R0',
 '2+AA': 'I2_L-1R0',
 '2+AC': 'I2_L-1R0',
 '2+AT': 'I2_L-1R0',
 '2+CA': 'I2_L-1R0',
 '2+CC': 'I2_L-1R0',
 '2+CG': 'I2_L-1R0',
 '2+GA': 'I2_L-1R0',
 '2+GC': 'I2_L-1R0',
 '2+GG': 'I2_L-1R0',
 '2+GT': 'I2_L-1R0',
 '2+TC': 'I2_L-1R0',
 '2+TG': 'I2_L-1R0',
 '2+TT': 'I2_L-1R0'}

# Correlate Observed and Predicted Outcomes

In [10]:
# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            predicted = data[t][method][target_site]["predicted"].astype(float) # Q
            observed = data[t][method][target_site]["actual"].astype(float)/sum(data[t][method][target_site]["actual"].astype(float)) # P
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/overall.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

Pearson's Correlation  KL Divergence  Jensen Shannon
Dataset     Method                                                           
FORECasT WT FORECasT                  0.808166       1.006983        0.473340
            Lindel                    0.704007       1.227826        0.485124
            X-CRISP-KLD               0.807170       0.947403        0.427136
            X-CRISP-MSE               0.809523       0.962321        0.422319
            inDelphi                  0.559852       1.567642        0.495571
inDelphi WT FORECasT                  0.850388       0.756300        0.422774
            Lindel                    0.782519       0.885720        0.419050
            X-CRISP-KLD               0.845209       0.701901        0.376567
            X-CRISP-MSE               0.847973       0.718534        0.373690
            inDelphi                  0.551572       1.080269        0.458516

In [11]:
# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    for method in models:
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            mh = np.array(data[t][method][target_site]["mh"])
            if (t not in BASELINE_TEST_FILES) and ("X-CRISP" in method):
                mh = np.array(list(mh) + ([False] * 21))

            if target_site in common_indels:
                common_indels[target_site] = np.intersect1d(common_indels[target_site], indels[mh])
            else:
                common_indels[target_site] = indels[mh]
    
    for method in models:
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            # deletions = np.array([not is_insertion(x, method) for x in indels])
            # mh = np.array(data[t][method][target_site]["mh"])
            mh = np.isin(indels, common_indels[target_site])
            predicted = data[t][method][target_site]["predicted"][mh].astype(float) # Q
            observed = data[t][method][target_site]["actual"][mh].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/mh.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

Pearson's Correlation  KL Divergence  Jensen Shannon
Dataset     Method                                                           
FORECasT WT FORECasT                  0.810218       0.726056        0.403390
            Lindel                    0.718656       0.938523        0.435969
            X-CRISP-KLD               0.816015       0.700319        0.379584
            X-CRISP-MSE               0.820142       0.709015        0.371023
            inDelphi                  0.644897       1.114311        0.468116
inDelphi WT FORECasT                  0.850355       0.535036        0.349633
            Lindel                    0.784875       0.663864        0.370221
            X-CRISP-KLD               0.854902       0.499115        0.325553
            X-CRISP-MSE               0.861207       0.496333        0.315144
            inDelphi                  0.600633       0.979540        0.458862

In [12]:
# t = '0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1'
# target_site = "0_0_0_0_AAATATCTTTAACCTAAAAC"
# test_f = file_mapping[t]
# common_indels = {}
# for method in models:
#     if method == "inDelphi": continue
#     for target_site in [target_site]:
#         indels = np.array(data[t][method][target_site]["indels"])
#         deletions = np.array([not is_insertion(x, method) for x in indels])
#         mhless = np.invert(np.array(data[t][method][target_site]["mh"]))
#         if (t not in BASELINE_TEST_FILES) and ("X-CRISP" in method):
#             mhless = np.array(list(mhless) + ([False] * 21))
#         mhless_deletions = deletions & mhless
#         if target_site in common_indels:
#             common_indels[target_site] = np.intersect1d(common_indels[target_site], indels[mhless_deletions])
#             if len(common_indels[target_site]) == 0:
#                 print(f"{target_site} {method} in common")
#                 print(indels[mhless_deletions])
#                 print(common_indels[target_site])
#         else:
#             common_indels[target_site] = indels[mhless_deletions]
#             if len(common_indels[target_site]) == 0:
#                 print(f"{target_site} {method} not in common")
#                 print(indels[mhless_deletions])
#                 print(common_indels[target_site])

In [13]:
def is_insertion(indel, method):
    if method in ["1NN", "KLD", "Lindel", "inDelphi"] or "transfer" in method:
        return indel in common_insertions or indel == "3" or indel == "3+X"
    if method == "FORECasT":
        return indel[0] == "I"

# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    for method in models:
        if method == "inDelphi": continue
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            deletions = np.array([not is_insertion(x, method) for x in indels])
            mhless = np.invert(np.array(data[t][method][target_site]["mh"]))
            if (t not in BASELINE_TEST_FILES) and ("X-CRISP" in method):
                mhless = np.array(list(mhless) + ([False] * 21))
            mhless_deletions = deletions & mhless
            if target_site in common_indels:
                common_indels[target_site] = np.intersect1d(common_indels[target_site], indels[mhless_deletions])
            else:
                common_indels[target_site] = indels[mhless_deletions]

    for method in models:
        if method == "inDelphi": continue
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"])
            mhless_deletions = np.isin(indels, common_indels[target_site])
            predicted = data[t][method][target_site]["predicted"][mhless_deletions].astype(float) # Q
            observed = data[t][method][target_site]["actual"][mhless_deletions].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)
            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
inf_oligos = df[~np.isfinite(df["KL Divergence"])].index.get_level_values(2)
df = df[~df.index.get_level_values(2).isin(inf_oligos)]
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/mhless.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

/var/folders/5x/n2td64g918lgvdh3qm2x3bhc0000gn/T/ipykernel_64480/3974721912.py:36: RuntimeWarning: invalid value encountered in divide
  observed = observed/sum(observed)


Pearson's Correlation  KL Divergence  Jensen Shannon
Dataset     Method                                                           
FORECasT WT FORECasT                  0.508695       2.375331        0.678785
            Lindel                    0.468319       2.310856        0.634852
            X-CRISP-KLD               0.475207       2.200699        0.624153
            X-CRISP-MSE               0.467629       2.239868        0.625448
inDelphi WT FORECasT                  0.623287       1.771776        0.620706
            Lindel                    0.582507       1.616749        0.560701
            X-CRISP-KLD               0.587618       1.485561        0.542779
            X-CRISP-MSE               0.578963       1.558970        0.548173

In [14]:
# copy takes about 14 seconds
ins_data = copy.deepcopy(data)

fasta_files = {
    "test": "../../src/data/FORECasT/test.fasta",
    "0105-mESC-Lib1-Cas9-Tol2-BioRep2-techrep1": "../../src/data/inDelphi/LibA.fasta"
}

guides = {}

for ff in list(fasta_files.values()):
    guides.update(get_details_from_fasta(ff))

ins_mapping = {}
t = "test"
all_1bp_insertions = np.array(["1+A", "1+C", "1+G", "1+T"])
for t in TEST_FILES:
    ins_mapping[t] = {}
    for o in common_oligos[t]:
        g = guides[o]
        cutsite = g["PAM Index"] - 3
        ins_mapping[t][o] = {}
        FORECasT_insertions = [i for i in data[t]["FORECasT"][o]["indels"] if "I1" in i]
        FORECasT_rep_insertions = [i for i in FORECasT_insertions if "C" in i]
        FORECasT_norep_insertions = [i for i in FORECasT_insertions if "C" not in i]
        if len(FORECasT_norep_insertions) != 1: print("wtf")
        for i in FORECasT_rep_insertions:
            _, L, C, R = re.split("L|C|R", i)
            rep_nuc = g["TargetSequence"][cutsite + int(R) -1]
            ins_mapping[t][o][i] = "1+{}".format(rep_nuc)
        for i in FORECasT_norep_insertions:
            ins_mapping[t][o][i] = list(np.setdiff1d(all_1bp_insertions, list(ins_mapping[t][o].values()))) 

ins_mapping[t][o]

def is_forecast_insertion(indel):
        return indel[:2] == "I1"

# collect predicted data into dataframe
rows = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    common_indels = {}
    # for method in ["inDelphi"]:
    for method in [m for m in models if m != "FORECasT"]:
        for target_site in common_oligos[t]:
            new_predicted = []
            new_observed = []
            new_indels = []
            indels = np.array(data[t][method][target_site]["indels"])
            predicted = pd.Series(list(data[t][method][target_site]["predicted"]), index=indels)
            observed = pd.Series(list(data[t][method][target_site]["actual"]), index=indels)

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            for i in ins_mapping[t][target_site]:
                new_predicted.append(predicted.loc[ins_mapping[t][target_site][i]].sum())
                new_observed.append(observed.loc[ins_mapping[t][target_site][i]].sum())
                new_indels.append(i)

            ins_data[t][method][target_site]["indels"] = np.array(new_indels)
            ins_data[t][method][target_site]["predicted"] = np.array(new_predicted)
            ins_data[t][method][target_site]["actual"] = np.array(new_observed)

    for method in ins_data[t].keys():
        for target_site in common_oligos[t]:
            indels = ins_data[t][method][target_site]["indels"]
            insertions = np.array([is_forecast_insertion(x) for x in indels])
            predicted = ins_data[t][method][target_site]["predicted"][insertions].astype(float) # Q
            observed = ins_data[t][method][target_site]["actual"][insertions].astype(float) # P

            predicted = predicted/sum(predicted)
            observed = observed/sum(observed)

            # then calculate
            indices.append((test_f, method, target_site))
            correlation = np.corrcoef(predicted, observed)[0,1]
            kl_divergence = entropy(observed, predicted)
            js = jensenshannon(observed, predicted)

            rows.append([correlation, kl_divergence, js])

indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method", "Target Site"])
df = pd.DataFrame(rows, index=indices, columns=["Pearson's Correlation", "KL Divergence", "Jensen Shannon"])
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/insertion.tsv", sep="\t")
df.groupby(["Dataset", "Method"]).mean()

/var/folders/5x/n2td64g918lgvdh3qm2x3bhc0000gn/T/ipykernel_64480/4136664625.py:75: RuntimeWarning: invalid value encountered in divide
  observed = observed/sum(observed)
/Users/colm/anaconda3/envs/xcrisp/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/colm/anaconda3/envs/xcrisp/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/var/folders/5x/n2td64g918lgvdh3qm2x3bhc0000gn/T/ipykernel_64480/4136664625.py:75: RuntimeWarning: invalid value encountered in divide
  observed = observed/sum(observed)
/Users/colm/anaconda3/envs/xcrisp/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/colm/anaconda3/envs/xcrisp/lib/python3.9/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
 

Pearson's Correlation  KL Divergence  Jensen Shannon
Dataset     Method                                                           
FORECasT WT FORECasT                  0.642682       0.238890        0.211644
            Lindel                    0.659909       0.256084        0.220513
            X-CRISP-KLD               0.659909       0.256084        0.220513
            X-CRISP-MSE               0.659909       0.256084        0.220513
            inDelphi                  0.574353       0.322935        0.260378
inDelphi WT FORECasT                  0.732948       0.152119        0.167314
            Lindel                    0.727372       0.182003        0.182233
            X-CRISP-KLD               0.727372       0.182003        0.182233
            X-CRISP-MSE               0.727372       0.182003        0.182233
            inDelphi                  0.689167       0.216672        0.205425

In [15]:
method = "X-CRISP-MSE"
indels = ins_data[t][method][target_site]["indels"]
insertions = np.array([is_forecast_insertion(x) for x in indels])
# predicted = ins_data[t][method][target_site]["predicted"][insertions].astype(float) # Q
# observed = ins_data[t][method][target_site]["actual"][insertions].astype(float) # P
np.array(ins_data[t][method][target_site]["indels"])[insertions]

array(['I1_L-1C1R1', 'I1_L-2C1R0', 'I1_L-1R0'], dtype='<U10')

In [16]:
from sklearn.metrics import precision_score, matthews_corrcoef, recall_score, f1_score


precisions = [.2, .3, .4, .5, .6, .7]
precision_X_mcc = {}

indices = []
rows_prec = []
rows_mcc = []
rows_recall = []
rows_f1_score = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():
    # for method in ["inDelphi"]:
        row_prec = []
        row_mcc = []
        row_recall = []
        row_f1_score = []
        for p in precisions:
            precision_X_preds = []
            precision_X_labels = []
            for target_site in common_oligos[t]:
                pred = np.array(data[t][method][target_site]["predicted"])
                x = sum((pred/sum(pred)) > p) == 1
                precision_X_preds.append(x)
                obs = np.array(data[t][method][target_site]["actual"])
                y = sum((obs/sum(obs)) > p) == 1
                precision_X_labels.append(y)
            row_prec.append(precision_score(precision_X_labels, precision_X_preds))
            row_mcc.append(matthews_corrcoef(precision_X_labels, precision_X_preds))
            row_recall.append(recall_score(precision_X_labels, precision_X_preds))
            row_f1_score.append(recall_score(precision_X_labels, precision_X_preds))
        print("Finished {}".format(method))
        rows_prec.append(row_prec)
        rows_mcc.append(row_mcc) 
        rows_recall.append(row_recall)
        rows_f1_score.append(row_f1_score)    
        indices.append((test_f, method))
        
        
  
indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method"])
df_mcc = pd.DataFrame(rows_mcc, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_prec = pd.DataFrame(rows_prec, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_recall = pd.DataFrame(rows_recall, index=indices, columns=["Precision-{}".format(p) for p in precisions])
df_f1_score = pd.DataFrame(rows_f1_score, index=indices, columns=["Precision-{}".format(p) for p in precisions])

df_mcc.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-mcc.tsv", sep="\t")
df_prec.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-prec.tsv", sep="\t")
df_recall.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-recall.tsv", sep="\t")
df_f1_score.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/precision-X-f1_score.tsv", sep="\t")

Finished X-CRISP-MSE
Finished X-CRISP-KLD
Finished inDelphi
Finished Lindel
Finished FORECasT
Finished X-CRISP-MSE
Finished X-CRISP-KLD


/Users/colm/anaconda3/envs/xcrisp/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Finished inDelphi
Finished Lindel
Finished FORECasT


# Summary Statistics Comparisons

In [17]:
from sklearn.metrics import mean_squared_error

# collect predicted data into dataframe
def is_1bp_deletion(indel, method):
    if method in ["X-CRISP-MSE", "X-CRISP-KLD", "Lindel", "FORECasT"]:
        return indel.split("+")[-1] == "1"
    if method == "inDelphi":
        return indel.split("+")[-1] == "1" or indel == "DL1"

def is_1bp_insertion(indel, method):
    if method in ["X-CRISP-MSE", "X-CRISP-KLD", "Lindel", "inDelphi"]:
        return indel in ['1+A', '1+C', '1+G', '1+T']
    if method == 'FORECasT':
        return indel[:2] == "I1"

def is_insertion(indel, method):
    if method in ["X-CRISP-MSE", "X-CRISP-KLD", "Lindel", "inDelphi"]:
        return indel in common_insertions or indel == "3" or indel == "3+X"
    if method == "FORECasT":
        return indel[0] == "I"

def is_frameshift(indel, method, t = "any"):
    length = None
    if method in ["X-CRISP-MSE", "X-CRISP-KLD", "Lindel"]:
        if is_insertion(indel, method):
            length = int(indel[0]) 
        else:
            length = int(indel.split("+")[-1]) 
    if method == "inDelphi":
        if is_insertion(indel, method):
            length = int(indel[0])
        elif indel[:2] == "DL":
            length = int(indel[2:])
        else:
            length = int(indel.split("+")[-1]) 
    if method == "FORECasT":
        if is_insertion(indel, method):
            length = int(indel.split("_")[0][1:])
        else:
            length = int(indel.split("+")[-1]) 
        
    allonebpframeshifts = np.array([1, 4, 7, 10, 13, 16, 19, 22, 25, 28])
    alltwobpframeshifts = allonebpframeshifts + 1

    if t == 1:
        return length in allonebpframeshifts
    elif t == 2:
        return length in alltwobpframeshifts
    else:
        return length % 3 != 0

rows_mse = []
rows_corr = []
indices = []
for t in TEST_FILES:
    test_f = file_mapping[t]
    for method in data[t].keys():

        # deletion frequency
        # one basepair deletions
        preddelfreq = []
        obsdelfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            dels = [not is_insertion(x, method) for x in indels]
            preddelratio = sum(np.array(data[t][method][target_site]["predicted"])[dels])/sum(data[t][method][target_site]["predicted"])
            preddelfreq.append(preddelratio)
            obsdelratio = sum(np.array(data[t][method][target_site]["actual"])[dels])/sum(data[t][method][target_site]["actual"])
            obsdelfreq.append(obsdelratio)
        delmse = mean_squared_error(preddelfreq, obsdelfreq)
        delcorr = np.corrcoef(preddelfreq, obsdelfreq)[0, 1]


        # one basepair deletions
        predonebpdelfreq = []
        obsonebpdelfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpdels = [is_1bp_deletion(x, method) for x in indels]
            predonebpdelratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpdels])/sum(data[t][method][target_site]["predicted"])
            predonebpdelfreq.append(predonebpdelratio)
            obsonebpdelratio = sum(np.array(data[t][method][target_site]["actual"])[onebpdels])/sum(data[t][method][target_site]["actual"])
            obsonebpdelfreq.append(obsonebpdelratio)
        onebpdelmse = mean_squared_error(predonebpdelfreq, obsonebpdelfreq)
        onebpdelcorr = np.corrcoef(predonebpdelfreq, obsonebpdelfreq)[0, 1]


        # one basepair insertions
        predonebpinsfreq = []
        obsonebpinsfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpins = [is_1bp_insertion(x, method) for x in indels]
            predonebpinsratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpins])/sum(data[t][method][target_site]["predicted"])
            predonebpinsfreq.append(predonebpinsratio)
            obsonebpinsratio = sum(np.array(data[t][method][target_site]["actual"])[onebpins])/sum(data[t][method][target_site]["actual"])
            obsonebpinsfreq.append(obsonebpinsratio)
        onebpinsmse = mean_squared_error(predonebpinsfreq, obsonebpinsfreq)
        onebpinscorr = np.corrcoef(predonebpinsfreq, obsonebpinsfreq)[0, 1]

        # one bp frameshift
        predonebpframeshiftfreq = []
        obsonebpframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            onebpframeshift = [is_frameshift(x, method, 1) for x in indels]
            predonebpframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[onebpframeshift])/sum(data[t][method][target_site]["predicted"])
            predonebpframeshiftfreq.append(predonebpframeshiftratio)
            obsonebpframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[onebpframeshift])/sum(data[t][method][target_site]["actual"])
            obsonebpframeshiftfreq.append(obsonebpframeshiftratio)
        onebpframeshiftmse = mean_squared_error(predonebpframeshiftfreq, obsonebpframeshiftfreq)
        onebpframeshiftcorr = np.corrcoef(predonebpframeshiftfreq, obsonebpframeshiftfreq)[0, 1]

        # two bp frameshift
        predtwobpframeshiftfreq = []
        obstwobpframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            twobpframeshift = [is_frameshift(x, method, 2) for x in indels]
            predtwobpframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[twobpframeshift])/sum(data[t][method][target_site]["predicted"])
            predtwobpframeshiftfreq.append(predtwobpframeshiftratio)
            obstwobpframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[twobpframeshift])/sum(data[t][method][target_site]["actual"])
            obstwobpframeshiftfreq.append(obstwobpframeshiftratio)
        twobpframeshiftmse = mean_squared_error(predtwobpframeshiftfreq, obstwobpframeshiftfreq)
        twobpframeshiftcorr = np.corrcoef(predtwobpframeshiftfreq, obstwobpframeshiftfreq)[0, 1]
        
        # frameshift
        predframeshiftfreq = []
        obsframeshiftfreq = []
        for target_site in common_oligos[t]:
            indels = np.array(data[t][method][target_site]["indels"]) 
            frameshift = [is_frameshift(x, method, "any") for x in indels]
            predframeshiftratio = sum(np.array(data[t][method][target_site]["predicted"])[frameshift])/sum(data[t][method][target_site]["predicted"])
            predframeshiftfreq.append(predframeshiftratio)
            obsframeshiftratio = sum(np.array(data[t][method][target_site]["actual"])[frameshift])/sum(data[t][method][target_site]["actual"])
            obsframeshiftfreq.append(obsframeshiftratio)
        frameshiftmse = mean_squared_error(predframeshiftfreq, obsframeshiftfreq)
        frameshiftcorr = np.corrcoef(predframeshiftfreq, obsframeshiftfreq)[0, 1]


        rows_mse.append([delmse, onebpdelmse, onebpinsmse, onebpframeshiftmse, twobpframeshiftmse, frameshiftmse])
        rows_corr.append([delcorr, onebpdelcorr, onebpinscorr, onebpframeshiftcorr, twobpframeshiftcorr, frameshiftcorr])
        indices.append((test_f, method))
  
indices = pd.MultiIndex.from_tuples(indices, names=["Dataset", "Method"])
df = pd.DataFrame(rows_mse, index=indices, columns=["Deletion", "1BP Deletion", "1BP Insertion", "1BP Frameshift", "2BP Frameshift", "Frameshift"])
df.groupby(["Dataset", "Method"])

df_corr = pd.DataFrame(rows_corr, index=indices, columns=["Deletion", "1BP Deletion", "1BP Insertion", "1BP Frameshift", "2BP Frameshift", "Frameshift"])


In [18]:
df.groupby(["Dataset", "Method"]).mean()

Deletion  1BP Deletion  1BP Insertion  \
Dataset     Method                                               
FORECasT WT FORECasT     0.010413      0.007034       0.009769   
            Lindel       0.010293      0.010404       0.008906   
            X-CRISP-KLD  0.009855      0.007365       0.007878   
            X-CRISP-MSE  0.009855      0.007376       0.007878   
            inDelphi     0.027646      0.018471       0.027646   
inDelphi WT FORECasT     0.005356      0.003379       0.004933   
            Lindel       0.006707      0.005058       0.005231   
            X-CRISP-KLD  0.006428      0.003342       0.005168   
            X-CRISP-MSE  0.006428      0.003283       0.005168   
            inDelphi     0.018991      0.018800       0.018991   

                         1BP Frameshift  2BP Frameshift  Frameshift  
Dataset     Method                                                   
FORECasT WT FORECasT           0.020279        0.015221    0.014817  
            Lindel             0.028070        0.023431    0.020499  
            X-CRISP-KLD        0.016482        0.012697    0.011623  
            X-CRISP-MSE        0.016033        0.012265    0.011178  
            inDelphi           0.035523        0.022578    0.020089  
inDelphi WT FORECasT           0.009132        0.007441    0.007262  
            Lindel             0.013382        0.011005    0.009541  
            X-CRISP-KLD        0.008690        0.007166    0.005824  
            X-CRISP-MSE        0.008345        0.007170    0.005807  
            inDelphi           0.030759        0.018499    0.015373

### CROTON comparisons

In [19]:
CROTON_F = "/Users/colm/repos/output/local/model_predictions/CROTON/{}_new.pkl"

summ_data = {}

for i, t in enumerate(TEST_FILES):
    croton_d = pkl.load(open(CROTON_F.format(t), 'rb'))
    summ_data[t] = croton_d

croton_i = pd.MultiIndex.from_product([[file_mapping[t] for t in TEST_FILES], ["CROTON"]], names=["Dataset", "Method"])


croton_del_freq_mse = []
croton_prob_1bpins_mse = []
croton_prob_1bpdel_mse = []
croton_one_bp_frameshift_mse = []
croton_two_bp_frameshift_mse = []
croton_frameshift_mse = []

for t in TEST_FILES:
    croton_del_freq_mse.append(mean_squared_error(summ_data[t]["predicted"]["del_freq"], summ_data[t]["actual"]["del_freq"]))
    croton_prob_1bpins_mse.append(mean_squared_error(summ_data[t]["predicted"]["prob_1bpins"], summ_data[t]["actual"]["prob_1bpins"]))
    croton_prob_1bpdel_mse.append(mean_squared_error(summ_data[t]["predicted"]["prob_1bpdel"], summ_data[t]["actual"]["prob_1bpdel"]))
    croton_one_bp_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["one_bp_frameshift"], summ_data[t]["actual"]["one_bp_frameshift"]))
    croton_two_bp_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["two_bp_frameshift"], summ_data[t]["actual"]["two_bp_frameshift"]))
    croton_frameshift_mse.append(mean_squared_error(summ_data[t]["predicted"]["frameshift"], summ_data[t]["actual"]["frameshift"]))

croton_mse_d = pd.DataFrame({
    "Deletion": croton_del_freq_mse,
    "1BP Insertion": croton_prob_1bpins_mse,
    "1BP Deletion": croton_prob_1bpdel_mse,
    "1BP Frameshift": croton_one_bp_frameshift_mse,
    "2BP Frameshift": croton_two_bp_frameshift_mse,
    "Frameshift": croton_frameshift_mse,
}, index=croton_i)

croton_mse_d

,,Deletion,1BP Insertion,1BP Deletion,1BP Frameshift,2BP Frameshift,Frameshift
Dataset,Method,,,,,,
FORECasT WT,CROTON,0.009358,0.008777,0.009544,0.045417,0.040763,0.046682
inDelphi WT,CROTON,0.003438,0.003261,0.005636,0.029145,0.026295,0.036946


In [20]:
df = pd.concat([df, croton_mse_d]).sort_index()
df.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/stats_comparison.tsv", sep="\t")
df

Deletion  1BP Deletion  1BP Insertion  \
Dataset     Method                                               
FORECasT WT CROTON       0.009358      0.009544       0.008777   
            FORECasT     0.010413      0.007034       0.009769   
            Lindel       0.010293      0.010404       0.008906   
            X-CRISP-KLD  0.009855      0.007365       0.007878   
            X-CRISP-MSE  0.009855      0.007376       0.007878   
            inDelphi     0.027646      0.018471       0.027646   
inDelphi WT CROTON       0.003438      0.005636       0.003261   
            FORECasT     0.005356      0.003379       0.004933   
            Lindel       0.006707      0.005058       0.005231   
            X-CRISP-KLD  0.006428      0.003342       0.005168   
            X-CRISP-MSE  0.006428      0.003283       0.005168   
            inDelphi     0.018991      0.018800       0.018991   

                         1BP Frameshift  2BP Frameshift  Frameshift  
Dataset     Method                                                   
FORECasT WT CROTON             0.045417        0.040763    0.046682  
            FORECasT           0.020279        0.015221    0.014817  
            Lindel             0.028070        0.023431    0.020499  
            X-CRISP-KLD        0.016482        0.012697    0.011623  
            X-CRISP-MSE        0.016033        0.012265    0.011178  
            inDelphi           0.035523        0.022578    0.020089  
inDelphi WT CROTON             0.029145        0.026295    0.036946  
            FORECasT           0.009132        0.007441    0.007262  
            Lindel             0.013382        0.011005    0.009541  
            X-CRISP-KLD        0.008690        0.007166    0.005824  
            X-CRISP-MSE        0.008345        0.007170    0.005807  
            inDelphi           0.030759        0.018499    0.015373

In [21]:
croton_del_freq_corr = []
croton_prob_1bpins_corr = []
croton_prob_1bpdel_corr = []
croton_one_bp_frameshift_corr = []
croton_two_bp_frameshift_corr = []
croton_frameshift_corr = []

for t in TEST_FILES:
    croton_del_freq_corr.append(np.corrcoef(summ_data[t]["predicted"]["del_freq"], summ_data[t]["actual"]["del_freq"])[0, 1])
    croton_prob_1bpins_corr.append(np.corrcoef(summ_data[t]["predicted"]["prob_1bpins"], summ_data[t]["actual"]["prob_1bpins"])[0, 1])
    croton_prob_1bpdel_corr.append(np.corrcoef(summ_data[t]["predicted"]["prob_1bpdel"], summ_data[t]["actual"]["prob_1bpdel"])[0, 1])
    croton_one_bp_frameshift_corr.append(np.corrcoef(summ_data[t]["predicted"]["one_bp_frameshift"], summ_data[t]["actual"]["one_bp_frameshift"])[0, 1])
    croton_two_bp_frameshift_corr.append(np.corrcoef(summ_data[t]["predicted"]["two_bp_frameshift"], summ_data[t]["actual"]["two_bp_frameshift"])[0, 1])
    croton_frameshift_corr.append(np.corrcoef(summ_data[t]["predicted"]["frameshift"], summ_data[t]["actual"]["frameshift"])[0, 1])

croton_corr_d = pd.DataFrame({
    "Deletion": croton_del_freq_corr,
    "1BP Insertion": croton_prob_1bpins_corr,
    "1BP Deletion": croton_prob_1bpdel_corr,
    "1BP Frameshift": croton_one_bp_frameshift_corr,
    "2BP Frameshift": croton_two_bp_frameshift_corr,
    "Frameshift": croton_frameshift_corr,
}, index=croton_i)

In [22]:
df_corr = pd.concat([df_corr, croton_corr_d]).sort_index()
df_corr.to_csv("/Users/colm/repos/x-crisp/data/processed/Performance/stats_comparison_corr.tsv", sep="\t")
df_corr

Deletion  1BP Deletion  1BP Insertion  \
Dataset     Method                                               
FORECasT WT CROTON       0.750063      0.146696       0.752872   
            FORECasT     0.748036      0.394244       0.752433   
            Lindel       0.713941      0.238706       0.744969   
            X-CRISP-KLD  0.716373      0.314595       0.756462   
            X-CRISP-MSE  0.716373      0.320689       0.756462   
            inDelphi     0.371831      0.055886       0.371831   
inDelphi WT CROTON       0.788096      0.155177       0.788153   
            FORECasT     0.690966      0.539252       0.699753   
            Lindel       0.718981      0.409850       0.720587   
            X-CRISP-KLD  0.715632      0.467589       0.723281   
            X-CRISP-MSE  0.715632      0.480576       0.723281   
            inDelphi     0.342396     -0.026277       0.342396   

                         1BP Frameshift  2BP Frameshift  Frameshift  
Dataset     Method                                                   
FORECasT WT CROTON             0.431919        0.375148    0.395120  
            FORECasT           0.794122        0.807794    0.823847  
            Lindel             0.639063        0.624639    0.665457  
            X-CRISP-KLD        0.806936        0.819856    0.834000  
            X-CRISP-MSE        0.811020        0.824963    0.838827  
            inDelphi           0.617103        0.665853    0.707376  
inDelphi WT CROTON             0.400082        0.321871    0.317181  
            FORECasT           0.826589        0.834339    0.856263  
            Lindel             0.721266        0.738143    0.760952  
            X-CRISP-KLD        0.825289        0.837203    0.867839  
            X-CRISP-MSE        0.834447        0.842375    0.871524  
            inDelphi           0.584106        0.634381    0.658699